In [1]:
from IPython.core.display import HTML
HTML(r"""
<style>
    .output-plaintext, .output-stream, .output {
        font-family: Courier; # Any monospaced font should work
        line-height: 1.3 !important;
        font-size: 14px !important;
    }
</style>
""")

# Basic math analysis for Hoot Owl Hoot
## Number of turns in a lost game
A game is lost when the sun token has moved 13 spaces.  This can only happen after at least 13 cards have been played.


### What is the probability that a player starts the game with at least one sun card?
Since the turn order is random, this probability is the same for all players.

$$
\begin{align*}

p\left(start\ with\ at\ least\ one\ sun\right) &= 1-p\left(start\ with\ zero\ suns\right) \\

&= 1 - \left( \frac{36}{50}*\frac{35}{49}*\frac{34}{48} \right) \\
&= 0.6357
\end{align*}
$$

In [2]:
# Probability of the first hand having a sun on the first turn
from math import factorial as fac
p = 1 - (fac(36)/fac(3)/fac(36-3)) / (fac(50)/fac(3)/fac(50-3))
print(f'The probability of the first hand being dealt at leat one sun is {p:.4f}')

The probability of the first hand being dealt at leat one sun is 0.6357


In [3]:
# Deal 100,000 games and count the number of times the first hand contains at least one sun
import game

n_games = 100_000
count = 0
for _ in range(n_games):
    g = game.Game()
    if game.SUN in g.hands[0]:
        count += 1

print(f'In {count / n_games:.2%} of the {n_games:,} games, the first hand started with at least one sun.')

In 63.68% of the 100,000 games, the first hand started with at least one sun.


## What is the expected position within the deck of the 13th sun card?
```
<---------------------------------k---------------->
       ^                          ^           ^  
       12 suns                    13th sun    1 sun
```

The position of the 13th sun has 12 suns before it and one sun after it.  The number of possible ways to achieve a configuration like this is the number of ways to configure 12 suns within $(k-1)$ positions multiplied by the number of positions that the last sun can be in, which is $(50-k)$.

$$ N_{specific\ permutations} = \binom{k-1}{12} \left( 50-k \right) $$

And the total number of ways to distribute 14 sun cards within 50 cards is calculated using a binomial coefficient.

$$ N_{total\ permutations} = \binom{50}{14} $$

So the probability that the 13th sun is in position $k$ is the ratio of the two expressions above.

$$ p(k) = \frac{\binom{k-1}{12} \left( 50-k \right)}{\binom{50}{14}} $$

And the expected position of the 13th sun is:

$$
\begin{align*}

E[k_{13}] &= \sum_{k=13}^{49}p(k)k \\

&= \sum_{k=13}^{49} \left[ \frac{\binom{k-1}{12} \left( 50-k \right)}{\binom{50}{14}} k \right] \\
&= \sum_{k=13}^{49} \left[ \frac{k(k-1)!(50-k)} {(k-13)!} \frac{14!*36!}{12!*50!} \right] \\
&= 44.2

\end{align*}
$$

In [4]:
# Set up 100,000 decks and find the average position of the 13th sun
n_games = 100_000
sum = 0

for _ in range(n_games):
    d = game.Deck()
    d.shuffle()
    sum += d.position_of_nth_card(game.SUN, 13)

avg = sum / n_games
print(f'The average position of the 13th sun is {avg + 1}')

The average position of the 13th sun is 44.20356


In [6]:
# Find the average length of a one player game
import agent

n_games = 100_000

for N_players in range(1,5):
    sum = 0

    for _ in range(n_games):
        a = agent.Agent(n_hands=N_players)
        a.play_game()
        sum += len(a.game.deck.cards)

    avg_cards_remaining = sum / n_games
    avg_cards_played = 50 - avg_cards_remaining - 3 * N_players + 1
    avg_cards_dealt = 50 - avg_cards_remaining

    print(f'The average cards played in a {N_players} player game using random moves is {avg_cards_played:.3f}.')
    print(f'The average number of cards dealt in a {N_players} player game using random moves is {avg_cards_dealt:.3f}')

The average cards played in a 1 player game using random moves is 42.195.
The average number of cards dealt in a 1 player game using random moves is 44.195
The average cards played in a 2 player game using random moves is 40.214.
The average number of cards dealt in a 2 player game using random moves is 45.214
The average cards played in a 3 player game using random moves is 38.153.
The average number of cards dealt in a 3 player game using random moves is 46.153
The average cards played in a 4 player game using random moves is 35.943.
The average number of cards dealt in a 4 player game using random moves is 46.943


This result confirms that a single player game is expected to have 44.2 cards dealt and have 42.2 moves.